In this exercise we tend to predict the age of individuals with respect to the image of their face by transfer learnign. 

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn 
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
import torch.nn.functional as F





In [5]:
# Transform on data
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.ColorJitter(),
        transforms.RandomVerticalFlip()
           ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    ]),
}

# getting data
data_dir = '/home/sepehr/neuralNetworks/Dataset/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_dataloader = dataloaders['train']
images, labels = next(iter(train_dataloader))
print(images.shape)
print(torch.cuda.is_available())

torch.Size([4, 3, 224, 224])
True


i put relu and dropout layer in my model for using structure as same as VGG classifier  

In [14]:
class NeuralNetwork(nn.Module) :
  def __init__(self, input_size, num_classes=5):
    super(NeuralNetwork, self).__init__()
    self.relu = nn.ReLU(inplace=True)
    self.dropout = nn.Dropout(p=0.5,inplace=False)
    self.fc = nn.Linear(in_features=input_size,out_features=5)

  def forward(self, x):

    x = self.relu(x)

    x = self.dropout(x)
    x = self.fc(x)

    return x

i set true req_grad of two conv layer for learning some feature.
i reach to this model of vgg with several test.

In [40]:
class CustomVggWithNeuralNetwork(nn.Module):
    def __init__(self, pretrained=True, num_classes=5):
        super(CustomVggWithNeuralNetwork, self).__init__()

        self.vgg = models.vgg16(pretrained=pretrained)
        for param in self.vgg.parameters():
            param.requires_grad = False
        for param in self.vgg.features[0:2].parameters():
            param.requires_grad = True
        for param in self.vgg.features[5:7].parameters():
            param.requires_grad = True
        self.base_network = NeuralNetwork(1000, num_classes)

    def forward(self, x):
        x = self.vgg(x)
        x = self.base_network(x)
        return x

In [33]:
combined_model = CustomVggWithNeuralNetwork(pretrained=True)
print(combined_model.eval)
# print(combined_model)
combined_model = combined_model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<bound method Module.eval of CustomVggWithNeuralNetwork(
  (vgg): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'test']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'test' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best test Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [34]:
# loss function
loss = nn.CrossEntropyLoss()
combined_model.zero_grad()
# optimizer
optimizer = optim.SGD(params=combined_model.parameters() , lr= 0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# training 
train_model(combined_model , loss , optimizer ,exp_lr_scheduler, num_epochs= 20)

Epoch 0/19
----------
train Loss: 2.1940 Acc: 0.1855
test Loss: 1.9343 Acc: 0.2400

Epoch 1/19
----------
train Loss: 2.3276 Acc: 0.1613
test Loss: 1.9702 Acc: 0.1600

Epoch 2/19
----------
train Loss: 2.0066 Acc: 0.2500
test Loss: 1.6199 Acc: 0.2400

Epoch 3/19
----------
train Loss: 1.8549 Acc: 0.2500
test Loss: 1.6759 Acc: 0.2800

Epoch 4/19
----------
train Loss: 1.6896 Acc: 0.2984
test Loss: 1.7129 Acc: 0.2400

Epoch 5/19
----------
train Loss: 1.6446 Acc: 0.3629
test Loss: 1.6190 Acc: 0.2400

Epoch 6/19
----------
train Loss: 1.7788 Acc: 0.2984
test Loss: 1.6374 Acc: 0.2000

Epoch 7/19
----------
train Loss: 1.8603 Acc: 0.2742
test Loss: 1.5866 Acc: 0.2400

Epoch 8/19
----------
train Loss: 1.6240 Acc: 0.3710
test Loss: 1.5769 Acc: 0.2800

Epoch 9/19
----------
train Loss: 1.5454 Acc: 0.3710
test Loss: 1.5642 Acc: 0.2400

Epoch 10/19
----------
train Loss: 1.4676 Acc: 0.3710
test Loss: 1.5747 Acc: 0.2800

Epoch 11/19
----------
train Loss: 1.6002 Acc: 0.3226
test Loss: 1.5682 Acc

CustomVggWithNeuralNetwork(
  (vgg): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=T

i reach very low acc beacause we have very very little data but my model show learn something.

In [35]:
correct = 0
total = 0
combined_model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    for data in dataloaders['test']:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = combined_model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 32.00%


### dataset_crop

in this section i crop face to decrease noise and reach higher acc. 

In [30]:
import os
import cv2

def crop_faces_folder(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Load the pre-trained face detector from OpenCV
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Iterate over all image files in the input folder
    for foldername in os.listdir(input_folder):
        for filename in os.listdir(os.path.join(input_folder, foldername)):

            if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):  # Add more extensions if needed
                # Load the image
                # print(foldername)
                image_path = os.path.join(input_folder,foldername, filename)
                image = cv2.imread(image_path)
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                # Detect faces in the image
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

                # Crop and save each face
                for i, (x, y, w, h) in enumerate(faces):
                    face_roi = image[y:y + h, x:x + w]
                    output_path = os.path.join(output_folder,foldername, f"{filename}")
                    cv2.imwrite(output_path, face_roi)

# Example usage
input_folder_train = '/home/sepehr/neuralNetworks/Dataset/train'
output_folder_train = '/home/sepehr/neuralNetworks/Dataset_crop/train'
input_folder_test = '/home/sepehr/neuralNetworks/Dataset/test'
output_folder_test = '/home/sepehr/neuralNetworks/Dataset_crop/test'
crop_faces_folder(input_folder_train, output_folder_train)
crop_faces_folder(input_folder_test, output_folder_test)


In [36]:
# Transform on data
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.ColorJitter(),
        transforms.RandomVerticalFlip()
           ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    ]),
}

# getting data
data_dir = '/home/sepehr/neuralNetworks/Dataset_crop/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_dataloader = dataloaders['train']
images, labels = next(iter(train_dataloader))
print(images.shape)
print(torch.cuda.is_available())

torch.Size([4, 3, 224, 224])
True


In [38]:
# loss function
loss = nn.CrossEntropyLoss()
combined_model.zero_grad()
# optimizer
optimizer = optim.SGD(params=combined_model.parameters() , lr= 0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# training 
train_model(combined_model , loss , optimizer ,exp_lr_scheduler, num_epochs= 20)

Epoch 0/19
----------
train Loss: 1.5922 Acc: 0.3429
test Loss: 1.8375 Acc: 0.1905

Epoch 1/19
----------
train Loss: 1.4481 Acc: 0.3714
test Loss: 1.7594 Acc: 0.2381

Epoch 2/19
----------
train Loss: 1.6336 Acc: 0.3714
test Loss: 1.8387 Acc: 0.1905

Epoch 3/19
----------
train Loss: 1.6222 Acc: 0.3429
test Loss: 1.9152 Acc: 0.1429

Epoch 4/19
----------
train Loss: 1.5564 Acc: 0.3619
test Loss: 1.7889 Acc: 0.1429

Epoch 5/19
----------
train Loss: 1.5052 Acc: 0.3524
test Loss: 1.7489 Acc: 0.3333

Epoch 6/19
----------
train Loss: 1.5061 Acc: 0.2667
test Loss: 1.8743 Acc: 0.2857

Epoch 7/19
----------
train Loss: 1.3311 Acc: 0.4476
test Loss: 1.7445 Acc: 0.3333

Epoch 8/19
----------
train Loss: 1.4718 Acc: 0.3714
test Loss: 1.6985 Acc: 0.3810

Epoch 9/19
----------
train Loss: 1.3884 Acc: 0.4000
test Loss: 1.6998 Acc: 0.3810

Epoch 10/19
----------
train Loss: 1.4192 Acc: 0.3810
test Loss: 1.6829 Acc: 0.3810

Epoch 11/19
----------
train Loss: 1.4264 Acc: 0.4762
test Loss: 1.6865 Acc

CustomVggWithNeuralNetwork(
  (vgg): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=T

with cropped image the acc increase 10% and this result show again my model learn something of this data

In [39]:
correct = 0
total = 0
combined_model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    for data in dataloaders['test']:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = combined_model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 42.86%
